# Top Campaign donors in Washington State

This notebook analyzes bulk data retrieved from [Open Secrets](http://www.opensecrets.org/myos/bulk.php) - specifically, donations to federal campaigns for the 2016 cycle through most of January.

Let's load the basic data and see what we've got.

In [1]:
import agate

table = agate.Table.from_csv('indivs_Washington16.csv')
print [c.name for c in table.columns]

[u'Cycle', u'Fectransid', u'Contribid', u'Contrib', u'Recipid', u'Orgname', u'Ultorg', u'Realcode', u'Date', u'Amount', u'Street', u'City', u'State', u'Zip', u'Recipcode', u'Type', u'CmteId', u'OtherID', u'Gender', u'Microfilm', u'Occupation', u'Employer', u'Source']


Next, let's make a pivot table off of it - just contributor name, their organization, their zip code, and how much they've given (taking into account that sum entries in the data account for refunds, i.e. negative amounts).

In [2]:
pivot = table.pivot(['Contrib', 'Orgname', 'Zip'], aggregation=agate.Sum('Amount')).order_by('Sum', reverse=True)

In [3]:
for row in pivot.order_by('Sum', reverse=True).limit(10).rows:
    print row['Contrib'], row['Orgname'], row['Zip'], row['Sum']

BALLMER, CONNIE E Partners for Our Children 98004 250000
PUYALLUP TRIBE OF INDIANS Puyallup Tribe of Indians 98404 186100
SCHWEITZER, BEATRIZ VALDEZ MRS Schweitzer Engineering Labs 99163 173000
SCHWEITZER, EDMUND O DR Schweitzer Engineering Labs 99163 173000
SCHWEITZER, BEATRIZ V Schweitzer Engineering Labs 99163 155000
SCHWEITZER, III, EDMUND O Schweitzer Engineering Labs 99163 150000
None Homemaker 99163 106200
None SCHWEITZER ENGINEERING LABS 99163 106200
SNOQUALMIE TRIBE Snoqualmie Tribe 98065 104800
MUCKLESHOOT INDIAN TRIBE Muckleshoot Indian Tribe 98092 75700


Now we need to be able to see what the contributor IDs mean, so we'll ask ProPublica via [their API](https://propublica.github.io/campaign-finance-api-docs).

In [4]:
import json
import os
import pycurl
from StringIO import StringIO

def get_cmte_from_id(cmte_id):
    api_key = os.getenv('PROPUBLICA_API_KEY')
    url = 'https://api.propublica.org/campaign-finance/v1/2016/committees/%s.json' % cmte_id

    buffer = StringIO()
    c = pycurl.Curl()
    c.setopt(c.URL, url)
    c.setopt(c.HTTPHEADER, ['X-API-KEY: %s' % api_key])
    c.setopt(c.WRITEDATA, buffer)
    c.perform()
    c.close()

    body = buffer.getvalue()
    # Body is a string in some encoding.
    # In Python 2, we can print it without knowing what the encoding is.
    try:
        return json.loads(body)['results'][0]
    except:
        return {}

Save this mapping to a file so we don't have to rebuild it later.

In [5]:
mapping = {}

for cmte in table.columns['CmteId'].values_distinct():
    mapping[cmte] = get_cmte_from_id(cmte)

print len(mapping)

776


In [6]:
with open('mapping.json', 'w+') as fh:
    fh.write(json.dumps(mapping))

Now annotate the table (and save it to `contribs_with_cmtes` when we're done).

In [7]:
with_cmtes = table.compute([
    ('CmteName', agate.Formula(agate.Text(), lambda k: mapping[k['CmteId']]['name'])),
    ('CmteParty', agate.Formula(agate.Text(), lambda k: mapping[k['CmteId']]['party'])),
    ('CmteAddress', agate.Formula(agate.Text(), lambda k: mapping[k['CmteId']]['address'])),
    ('CmteCity', agate.Formula(agate.Text(), lambda k: mapping[k['CmteId']]['city'])),
    ('CmteState', agate.Formula(agate.Text(), lambda k: mapping[k['CmteId']]['state'])),
    ('CmteZip', agate.Formula(agate.Text(), lambda k: mapping[k['CmteId']]['zip'])),
    ('CmteType', agate.Formula(agate.Text(), lambda k: mapping[k['CmteId']]['committee_type'])),
])

In [8]:
with_cmtes.to_csv('contribs_with_cmtes.csv')

What committees raised the most money from Washington?

In [9]:
top_cmtes_pivot = with_cmtes.pivot(['CmteName'], aggregation=agate.Sum('Amount')).order_by('Sum', reverse=True)

In [10]:
for row in top_cmtes_pivot.order_by('Sum', reverse=True).limit(10).rows:
    print '%s raised %d' % (row['CmteName'], row['Sum'])

PEOPLE FOR PATTY MURRAY raised 983429
HILLARY FOR AMERICA raised 860582
NRCC raised 763246
DSCC raised 572685
MURRAY VICTORY FUND raised 498006
ACTBLUE raised 414294
REPUBLICAN NATIONAL COMMITTEE raised 385715
BERNIE 2016 raised 384694
PEOPLE FOR DEREK KILMER raised 354832
WASHINGTON STATE REPUBLICAN PARTY raised 340495


Let's break out just presidential committees.

In [11]:
pres_only = with_cmtes.where(lambda k: k['CmteType'] == 'P')

Who were the top donors to presidential committees? (Save the full table to `top_pres_donors.csv`)

In [12]:
top_pres_donors_pivot = pres_only.pivot(
    ['Contrib', 'CmteName'], aggregation=agate.Sum('Amount')).order_by('Sum', reverse=True)
top_pres_donors_pivot.limit(10).print_table()
top_pres_donors_pivot.to_csv('top_pres_donors.csv')

|-----------------------+----------------------+---------|
|  Contrib              | CmteName             |    Sum  |
|-----------------------+----------------------+---------|
|  GAMORAN, SAUL        | CRUZ FOR PRESIDENT   | 10,800  |
|  MEISENBACH, JOHN     | CRUZ FOR PRESIDENT   | 10,000  |
|  LANDON, JOSEPH PA... | JEB 2016, INC.       |  8,100  |
|  TUCKER, ROBERT       | JEB 2016, INC.       |  8,100  |
|  ERWIN, GERALD        | CARSON AMERICA       |  7,700  |
|  NEUPERT, SHERYL S... | MARCO RUBIO FOR P... |  5,400  |
|  JULIAN, REBECCA L    | CRUZ FOR PRESIDENT   |  5,400  |
|  RICE, MERIDEL        | CRUZ FOR PRESIDENT   |  5,400  |
|  SABOL, JOHN          | HILLARY FOR AMERICA  |  5,400  |
|  KETCHAM, SAM         | HILLARY FOR AMERICA  |  5,400  |
|-----------------------+----------------------+---------|


What were the top parties by fundraising, and how much did they get? (Save the full table to `top_parties.csv`)

In [13]:
top_parties = with_cmtes.pivot(['Contrib', 'CmteParty'], aggregation=agate.Sum('Amount')).order_by('Sum', reverse=True)
top_parties.limit(10).print_table()
top_parties.to_csv('top_parties.csv')

|-----------------------+-----------+----------|
|  Contrib              | CmteParty |     Sum  |
|-----------------------+-----------+----------|
|  BALLMER, CONNIE E    |           | 250,000  |
|                       | REP       | 212,400  |
|  SCHWEITZER, BEATR... | REP       | 173,000  |
|  SCHWEITZER, EDMUN... | REP       | 173,000  |
|  SCHWEITZER, BEATR... |           | 155,000  |
|  SCHWEITZER, III, ... |           | 150,000  |
|  PUYALLUP TRIBE OF... | DEM       | 130,500  |
|  KEITH, MICHAEL       | DEM       |  93,405  |
|  CLARK, JOSEPH        | REP       |  72,000  |
|  MUCKLESHOOT INDIA... | DEM       |  70,500  |
|-----------------------+-----------+----------|


What were the largest donor-party pairs? I.e., which donors gave the most to whom? (Save the full table to `top_cmte_donors.csv`)

In [14]:
top_cmte_donors = with_cmtes.pivot(
    ['Contrib', 'CmteName', 'CmteParty'], aggregation=agate.Sum('Amount')).order_by('Sum', reverse=True)
top_cmte_donors.limit(10).print_table()
top_cmte_donors.to_csv('top_cmte_donors.csv')

|-----------------------+----------------------+-----------+----------|
|  Contrib              | CmteName             | CmteParty |     Sum  |
|-----------------------+----------------------+-----------+----------|
|  BALLMER, CONNIE E    | AMERICANS FOR RES... |           | 250,000  |
|                       | NRCC                 | REP       | 212,400  |
|  SCHWEITZER, BEATR... | MCMORRIS RODGERS ... |           | 150,000  |
|  SCHWEITZER, III, ... | MCMORRIS RODGERS ... |           | 150,000  |
|  SCHWEITZER, BEATR... | NRCC                 | REP       | 139,600  |
|  SCHWEITZER, EDMUN... | NRCC                 | REP       | 139,600  |
|  CLARK, JOSEPH        | REPUBLICAN NATION... | REP       |  66,600  |
|  PUYALLUP TRIBE OF... | DSCC                 | DEM       |  64,800  |
|  MCCAW, CRAIG MR      | RIGHT TO RISE USA    |           |  50,000  |
|  LANDON, JOSEPH MR    | RIGHT TO RISE USA    |           |  50,000  |
|-----------------------+----------------------+-----------+----